In [1]:
import boto3
from botocore.exceptions import ClientError
from types_boto3_dynamodb.service_resource import Table
dynamodb=boto3.resource("dynamodb")
resource:Table=dynamodb.Table("practice")

put item

In [13]:
try:
    response=resource.put_item(
        Item={
            "pk":"2",
            "sk":"details"
        }
    )
    
except  ClientError as e:
    print(type(e))
    print(str(e))

Get Item

In [ ]:
response=resource.get_item(
    Key={
        "pk":"1",
        "sk":"details"
    }
)
print(response)

In [ ]:
response.get("Item")

query item

In [3]:
from boto3.dynamodb.conditions import Key
response=resource.query(
    KeyConditionExpression=Key("pk").eq("1") & Key("sk").begins_with("PROF"),
    
)
print(response)

{'Items': [{'pk': '1', 'sk': 'PROFILE1'}, {'pk': '1', 'sk': 'PROFILE2'}], 'Count': 2, 'ScannedCount': 2, 'ResponseMetadata': {'RequestId': 'ADSNPPK1S91B0L16JNEBSA8C17VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Jan 2026 14:45:08 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '116', 'connection': 'keep-alive', 'x-amzn-requestid': 'ADSNPPK1S91B0L16JNEBSA8C17VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1725161946'}, 'RetryAttempts': 0}}


In [4]:
print(response.get("Items"))

[{'pk': '1', 'sk': 'PROFILE1'}, {'pk': '1', 'sk': 'PROFILE2'}]


query with partiql

In [11]:

client = resource.meta.client

stmt = "SELECT pk, sk, name FROM \"practice\" WHERE pk='1' AND begins_with(sk, 'PROF')"

partiql_resp = client.execute_statement(
    Statement=stmt
)
print(partiql_resp)

{'Items': [{'pk': '1', 'sk': 'PROFILE1'}, {'pk': '1', 'sk': 'PROFILE2'}], 'ResponseMetadata': {'RequestId': 'KGG1HQCULSTU2NR4FANM3L9VGFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Jan 2026 14:50:40 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '89', 'connection': 'keep-alive', 'x-amzn-requestid': 'KGG1HQCULSTU2NR4FANM3L9VGFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '799767965'}, 'RetryAttempts': 0}}


In [12]:

partiql_resp.get("Items")

[{'pk': '1', 'sk': 'PROFILE1'}, {'pk': '1', 'sk': 'PROFILE2'}]

Batch get

In [16]:
client = resource.meta.client

request_items = {
    "practice": {
        "Keys": [
            {"pk":'1', "sk": "details"},
            {"pk": "2", "sk":  "details"},
        ],
    }
}

resp = client.batch_get_item(RequestItems=request_items)
print(resp)

{'Responses': {'practice': [{'pk': '1', 'name': 'rohith', 'sk': 'details'}, {'pk': '2', 'sk': 'details'}]}, 'UnprocessedKeys': {}, 'ResponseMetadata': {'RequestId': 'V9SE522NG1M0DJ7URDFOVD9QBFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Jan 2026 15:07:40 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '147', 'connection': 'keep-alive', 'x-amzn-requestid': 'V9SE522NG1M0DJ7URDFOVD9QBFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1132393520'}, 'RetryAttempts': 0}}


In [17]:
items = resp.get("Responses", {}).get("practice", [])

while resp.get("UnprocessedKeys"):
    resp = client.batch_get_item(RequestItems=resp["UnprocessedKeys"])
    items.extend(resp.get("Responses", {}).get("practice", []))

items

[{'pk': '1', 'name': 'rohith', 'sk': 'details'}, {'pk': '2', 'sk': 'details'}]

batch get partiql


In [18]:
statements=[
    {
        "Statement":"Select * FROM \"practice\" where pk='1' and sk='details'"
    },
    {
        "Statement":"Select * FROM \"practice\" where pk='2' and sk='details'"
    }
    
]
resp=client.batch_execute_statement(Statements=statements)
print(resp)

{'Responses': [{'TableName': 'practice', 'Item': {'pk': '1', 'name': 'rohith', 'sk': 'details'}}, {'TableName': 'practice', 'Item': {'pk': '2', 'sk': 'details'}}], 'ResponseMetadata': {'RequestId': 'P0CTKD1NPO263327NJIR6ODC0FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Jan 2026 15:16:35 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '177', 'connection': 'keep-alive', 'x-amzn-requestid': 'P0CTKD1NPO263327NJIR6ODC0FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3106462684'}, 'RetryAttempts': 0}}


In [20]:
items = resp.get("Responses", {})
for item in items:
    print(item.get("Item"))

{'pk': '1', 'name': 'rohith', 'sk': 'details'}
{'pk': '2', 'sk': 'details'}


batch write

In [25]:
client = resource.meta.client

response = client.batch_write_item(
    RequestItems={
        "practice": [
            {
                "PutRequest": {
                    "Item": {
                        "pk": "3",
                        "sk": "details",
                        "name":"ganesh"
                    }
                }
            },
            {
                "PutRequest": {
                    "Item": {
                        "pk": "4",
                        "sk": "details",
                        "name":"suresh"
                    }
                }
            },
        ]
    }
)

print(response)



{'UnprocessedItems': {}, 'ResponseMetadata': {'RequestId': 'MTSLH83TSULEV0V5K21G36G10NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Jan 2026 15:42:25 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '23', 'connection': 'keep-alive', 'x-amzn-requestid': 'MTSLH83TSULEV0V5K21G36G10NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '4185382651'}, 'RetryAttempts': 0}}


partiql batchwrite

In [36]:
client = resource.meta.client

statements = [
    {
        "Statement": "INSERT INTO \"practice\" VALUE {'pk':'8','sk':'details','name':'rahul'}"
    },
    {
        "Statement":"INSERT INTO \"practice\" VALUE {'pk':'9','sk':'details','name':'roshan'}"
    }
]

resp = client.batch_execute_statement(Statements=statements)
print(resp)

{'Responses': [{'TableName': 'practice'}, {'TableName': 'practice'}], 'ResponseMetadata': {'RequestId': 'RFMTDA534MD07CML9ETJKPHT5FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 05 Jan 2026 15:55:44 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '65', 'connection': 'keep-alive', 'x-amzn-requestid': 'RFMTDA534MD07CML9ETJKPHT5FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2751026947'}, 'RetryAttempts': 0}}


conditional expression with put

In [39]:
try:
    item={
        "pk":"1",
        "sk":"details"
    }

    resource.put_item(
        Item=item,
        ConditionExpression="attribute_not_exists(pk)"
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'ConditionalCheckFailedException':
        print("Item already exists, not overwritten.")
    else:
        raise e

Item already exists, not overwritten.


transaction

In [42]:
client = resource.meta.client
try:
    client.transact_write_items(
        TransactItems=[
            {
                "Put": {
                    "TableName": "practice",
                    "Item": {
                        "pk": "10",
                        "sk": "details",
                    },
                }
            },
            {
                "Put": {
                    "TableName": "practice",
                    "Item": {
                        "pk": "10",
                        "sk": "PROFILE",
                    },
                }
            },
        ]
    )
except ClientError as err:
    raise
